Importing

In [ ]:
import torch
from torch.optim import Adam
from torch.optim import LBFGS
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from scipy import integrate # numerical integration tool for ode's

Physical System to Model: Spring, Mass, and Damper System

$ F(t) -c \frac{d\mathbf{x}}{d t}(t) - kx(t) = m\frac{d\mathbf{x^2}}{d t}(t)$

$\frac{d\mathbf{x^2}}{d t}(t) = ( F(t) -c \frac{d\mathbf{x}}{d t}(t) - kx(t) ) /m$

In [ ]:
# Physical Parameters, c = damping constant, k =  stiffness of spring, m = mass.
p = dict(c=4, k=2, m=10, F=0, x0=1, dx0dt=0)
p['d2x0dt2'] = (p['F'] - p['c']*p['dx0dt'] - p['k']*p['x0']) / p['m']
#initial conditions
x0 = [p['x0'],p['dx0dt']]
#plot to time T
T = 30

def f(x,t,p):
    x1, x2 = x[0], x[1]
    dx1 = x2
    dx2 = (p['F'] - p['c']*x2 - p['k']*x1) / p['m']
    return([dx1,dx2])

#Function to solve the physical system using numerical integration tool
def run_oscil(x0,T,p):
    t = np.linspace(0, T, 1000)
    dxdt = lambda x,t : f(x, t, p)
    x_t = integrate.odeint(dxdt, x0, t)
    return(t,x_t)

#Plotting neumerical solution
t_plot, x_plot = run_oscil(x0,T,p)
plt.plot(t_plot,x_plot)
plt.xlabel('time [s]')
#x_1 is position, x_2 is velocity
plt.legend(['$x_1(t)$','$x_2(t)$'])
plt.title("Numerical Solution to Spring, Mass, and Damper system")


The equation that models the Spring, Mass, and Damper System is the following:
$ F(t) = m\frac{d\mathbf{x^2}}{d t}(t) + c \frac{d\mathbf{x}}{d t}(t) + kx(t)$

The following parameters are needed to fully define the system:
1. Sum of Forces (F)
2. Mass (m)
3. Damping Constant (c)
4. Spring Stiffness (k)
5. Time (t)
6. Initial Conditions (x0)

These parameters can be used to determine the position (x) at current time (t).

Creating vectors for all parameters to be modelled

In [ ]:
train_tuple = tuple()
# Range of t to be modelled [0,30]S
t_train = torch.linspace(0,30,64).view(-1,1)
train_tuple += (t_train,)

#setting other variables as constants
F_train = torch.ones_like(t_train)*p['F']
#train_tuple += (F_train,)
m_train = torch.ones_like(t_train)*p['m']
#train_tuple += (m_train,)
c_train = torch.ones_like(t_train)*p['c']
#train_tuple += (c_train,)
k_train = torch.ones_like(t_train)*p['k']
#train_tuple += (k_train,)
x0_train = torch.ones_like(t_train)*p['x0']
#train_tuple += (x0_train,)
dx0dt_train = torch.ones_like(t_train)*p['dx0dt']
#train_tuple += (dx0dt_train,)
d2x0dt2_train = torch.ones_like(t_train)*p['d2x0dt2']
#train_tuple += (d2x0dt2_train,)

train = torch.cat(train_tuple,1).requires_grad_(True)

Defining the Model, Loss and Optimizer

In [ ]:
# len i/o nodes, i_len nodes per layer and Tanh activation fn, wide and shallow (64x4)
i_len = 64

#params to take and approximate
params_in = 1
params_out = 1

model = nn.Sequential(
    nn.Linear(params_in, i_len),
    
    nn.Tanh(),
    nn.Linear(i_len, i_len),
    nn.Tanh(),
    nn.Linear(i_len, i_len),
    nn.Tanh(),

    nn.Linear(i_len, params_out)
    )

# choose optimizer
optim = LBFGS(model.parameters(),history_size=15)    

Creating a custom loss function using the following:

Physics Loss:

$ F(t) - m\frac{d\mathbf{x^2}}{d t}(t) - c \frac{d\mathbf{x}}{d t}(t) - kx(t) = 0$

Initial Conditions Loss:

$x(0) = x_0$

$\frac{d\mathbf{x}}{d t}(0) = \frac{d\mathbf{x_0}}{d t}(t)$

$\frac{d\mathbf{x^2}}{d t}(0) = ( F(t) -c \frac{d\mathbf{x}}{d t}(0) - kx(0) ) /m$

In [ ]:
def loss_fn(model, inputs):
    # model: model
    # batch: vector with cols 0:t, 1:F, 2:M, 3:c, 4:k, 5:x(0), 6:x'(0), 7:x''(0)

    # Obtaining parameters to compute loss.
    x_pred = model(inputs)

    first_deriv = torch.autograd.grad(outputs=x_pred, inputs=inputs, grad_outputs=torch.ones_like(x_pred), create_graph=True, retain_graph=True)[0]
    second_deriv = torch.autograd.grad(outputs=first_deriv, inputs=inputs, grad_outputs=torch.ones_like(first_deriv), retain_graph=True)[0]
    dxdt_pred = first_deriv[:, 0]
    d2xdt2_pred = second_deriv[:, 0]

    F = p['F']
    #F = inputs[:,1]
    m = p['m']
    #m = inputs[:,2]
    c = p['c']
    #c = inputs[:,3]
    k = p['k']
    #k = inputs[:,4]

    # Calculate loss based on governing physics eqns
    phy_loss = torch.mean( (F - m*d2xdt2_pred - c*dxdt_pred - k*x_pred)**2 )

    # Calulate loss from boundary conditions
    initial_input = inputs.detach().clone()
    initial_input[:,0] = torch.zeros_like(boundary_input[:,0])
    initial_input.requires_grad_(True)
    
    x0_pred = model(initial_input)
    dx0dt_pred = torch.autograd.grad(x0_pred, initial_input, torch.ones_like(x0_pred), create_graph=True, retain_graph=True)[0]
    d2x0dt2_pred = torch.autograd.grad(dx0dt_pred, initial_input, torch.ones_like(dx0dt_pred), retain_graph=True)[0]

    x0 = p['x0']
    #x0 = initial_input[:,5]
    dx0dt = p['dx0dt']
    #dx0dt = initial_input[:,6]
    d2x0dt2 = p['d2x0dt2']
    #d2x0dt2 = initial_input[:,7]

    x0_loss = torch.mean( (x0_pred.view(-1,1) - x0)**2 )
    dx0dt_loss = torch.mean( (dx0dt_pred.view(-1,1) - dx0dt)**2 )
    d2x0dt2_loss = torch.mean( (d2x0dt2_pred.view(-1,1) - d2x0dt2)**2 )
    i_loss = x0_loss + dx0dt_loss + d2x0dt2_loss

    loss = phy_loss + i_loss

    return loss

Train...

In [ ]:
n_epochs = 3000
prev_loss = list()

def closure():
    loss = loss_fn(model, train)
    optim.zero_grad()
    loss.backward()
    prev_loss.append(loss.item())
    return loss

for epoch in range(n_epochs):
    optim.step(closure)
    if not (epoch%100):
        print(f'Finished epoch {epoch}, latest loss {prev_loss[-1]:.2E}')
        prev_loss = list()

Comparing plots of MLP approximation against numerical solution.

In [ ]:
plt.plot(t_plot,x_plot[:,0],label='Numerical Solution')
plt.xlabel('time [s]')
plt.title("Comparing MLP Approximation and Numerical Solution")

# obtaining plotting information
# inputs: vector with cols 0:t, 1:F, 2:M, 3:c, 4:k, 5:x(0), 6:x'(0)

#creating tensors to be plotted
plot_tuple = tuple()

plot_t = torch.from_numpy(t_plot).float().view(-1,1)
plot_tuple += (plot_t,)
plot_F = torch.ones_like(plot_t)*p['F']
#plot_tuple += (plot_F,)
plot_m = torch.ones_like(plot_t)*p['m']
#plot_tuple += (plot_m,)
plot_c = torch.ones_like(plot_t)*p['c']
#plot_tuple += (plot_c,)
plot_k = torch.ones_like(plot_t)*p['k']
#plot_tuple += (plot_k,)
plot_x0 = torch.ones_like(plot_t)*p['x0']
#plot_tuple += (plot_x0,)
plot_dx0dt = torch.ones_like(plot_t)*p['dx0dt']
#plot_tuple += (plot_dx0dt,)
plot_d2x0dt2 = torch.ones_like(plot_t)*p['d2x0dt2']
#plot_tuple += (plot_d2x0dt2,)

plot_input = torch.cat(plot_tuple, 1)
pred = model(plot_input)
pred_plot = torch.reshape(pred, (-1,)).detach().numpy()

plt.plot(t_plot,pred_plot,label='MLP Approximation')
plt.legend()